In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import torch
import easyocr

# License Plate Recognition functions
def detect_license_plate(frame, model):
    frame = [frame]
    results = model(frame)
    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    return labels, cordinates

def plot_license_plate_boxes(results, frame, classes):
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    for i in range(n):
        row = cord[i]
        if row[4] >= 0.55:
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
            text_d = classes[int(labels[i])]
            coords = [x1,y1,x2,y2]
            plate_num = recognize_plate_easyocr(img = frame, coords= coords, reader= EASY_OCR, region_threshold= OCR_TH)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255,0), -1)
            cv2.putText(frame, f"{plate_num}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255), 2)

    return frame

def recognize_plate_easyocr(img, coords, reader, region_threshold):
    xmin, ymin, xmax, ymax = coords
    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)]
    ocr_result = reader.readtext(nplate)
    text = filter_text(region=nplate, ocr_result=ocr_result, region_threshold= region_threshold)

    if len(text) ==1:
        text = text[0].upper()
        #text1=image_path_dehaze+" "+''.join(text)
    with open("C:/Users/welcome/Desktop/outputText/output_txt.txt", 'a') as f:
            f.write('\n'+''.join(text))

    return text

def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    plate = []
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate

# Image Dehazing functions
def dark_channel(image, patch_size):
    b, g, r = cv2.split(image)
    min_channel = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch_size, patch_size))
    dark = cv2.erode(min_channel, kernel)
    return dark

def estimate_atmospheric_light(image, dark_channel, percentage):
    pixels = dark_channel.size
    flat_dark = dark_channel.flatten()
    sorted_dark = np.sort(flat_dark)
    threshold_index = int(pixels * percentage / 100)
    atmospheric_light = np.mean(sorted_dark[-threshold_index:])
    return atmospheric_light

def estimate_transmission(image, atmospheric_light, omega):
    epsilon = 1e-6
    transmission = np.ones_like(image, dtype=np.float32)
    transmission[:, :, 0] = atmospheric_light / (image[:, :, 0] + epsilon)
    transmission[:, :, 1] = atmospheric_light / (image[:, :, 1] + epsilon)
    transmission[:, :, 2] = atmospheric_light / (image[:, :, 2] + epsilon)
    return 1 - omega * dark_channel(transmission, 15)

def dehaze(image, transmission, atmospheric_light, t0):
    dehazed = np.zeros_like(image, dtype=np.float32)
    for i in range(3):
        dehazed[:, :, i] = (image[:, :, i] - atmospheric_light) / np.maximum(transmission, t0) + atmospheric_light
    return np.clip(dehazed, 0, 255).astype(np.uint8)

def dcp_dehazing(image, patch_size=15, percentage=0.1, omega=0.9, t0=0.1):
    dark = dark_channel(image, patch_size)
    atmospheric_light = estimate_atmospheric_light(image, dark, percentage)
    transmission = estimate_transmission(image, atmospheric_light, omega)
    return dehaze(image, transmission, atmospheric_light, t0)

# GUI functions
def select_image_lp():
    global image_path_lp

    image_path_lp = filedialog.askopenfilename()

    if image_path_lp:
        frame = cv2.imread(image_path_lp)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = detect_license_plate(frame, model=model)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = plot_license_plate_boxes(results, frame, classes=classes)

        img = Image.fromarray(frame)
        img = img.resize((800, 600), Image.BICUBIC)
        img = ImageTk.PhotoImage(img)
        label_lp.config(image=img)
        label_lp.image = img

def select_image_dehaze():
    global image_path_dehaze

    image_path_dehaze = filedialog.askopenfilename()

    if image_path_dehaze:
        image = cv2.imread(image_path_dehaze)
        if image is None:
            print("Error: Unable to load the image from the provided path.")
        else:
            dehazed_image = dcp_dehazing(image)

            original_image = Image.open(image_path_dehaze)
            original_image = original_image.resize((700, 700))
            original_image = ImageTk.PhotoImage(original_image)
            original_image_label.config(image=original_image)
            original_image_label.image = original_image

            dehazed_image = Image.fromarray(cv2.cvtColor(dehazed_image, cv2.COLOR_BGR2RGB))
            dehazed_image = dehazed_image.resize((700, 700))
            dehazed_image = ImageTk.PhotoImage(dehazed_image)
            dehazed_image_label.config(image=dehazed_image)
            dehazed_image_label.image = dehazed_image

# Load the YOLO model
model = torch.hub.load('./yolov5', 'custom', source='local', path='C:/Users/welcome/Desktop/yolov5-master/yolov5/runs/train/exp35/weights/best.pt', force_reload=True)
classes = model.names
EASY_OCR = easyocr.Reader(['en'])
OCR_TH = 0.2

# Create main window
root = tk.Tk()
root.title("Combined Application")

# Create a notebook (tabs)
notebook = ttk.Notebook(root)

# License Plate Recognition tab
tab_lp = ttk.Frame(notebook)
notebook.add(tab_lp, text='License Plate Recognition')

label_lp = tk.Label(tab_lp)
label_lp.pack()

select_image_button_lp = tk.Button(tab_lp, text="Select Image", command=select_image_lp,width=20, height=3)
select_image_button_lp.pack(pady=10)

# Image Dehazing tab
tab_dehaze = ttk.Frame(notebook)
notebook.add(tab_dehaze, text='Image Dehazing')

original_image_label = tk.Label(tab_dehaze)
original_image_label.grid(row=0, column=0, padx=10, pady=10, rowspan=2)

dehazed_image_label = tk.Label(tab_dehaze)
dehazed_image_label.grid(row=0, column=1, padx=10, pady=10, rowspan=2)

select_image_button_dehaze = tk.Button(tab_dehaze, text="Select Image", command=select_image_dehaze, width=20, height=3)
select_image_button_dehaze.grid(row=2, column=0, columnspan=2, pady=10, sticky="nsew")

# Configure grid to make the button center-aligned
tab_dehaze.grid_rowconfigure(2, weight=1)
tab_dehaze.grid_columnconfigure(0, weight=1)
tab_dehaze.grid_columnconfigure(1, weight=1)

notebook.pack(expand=True, fill='both')
root.mainloop()
